In [1]:
pip install psycopg2

  Using cached psycopg2-2.9.9-cp310-cp310-linux_x86_64.whl
Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import psycopg2
import psycopg2.extras
import boto3
import random
import time

bedrock = boto3.client('bedrock-runtime')

In [2]:
[models['modelId'] for models in bedrock.list_foundation_models()['modelSummaries']]

AttributeError: 'BedrockRuntime' object has no attribute 'list_foundation_models'

In [2]:
connection = psycopg2.connect(dbname='******', user='*****', password='*****', host='*****', port='*****', sslmode='*****')

query = "SELECT nombre FROM public.articulos_dev__articulos WHERE empresa_id = 279 ORDER BY id ASC LIMIT 20"
cursor = connection.cursor()
cursor.execute(query)
results = cursor.fetchall()
for item in results:
    print(item)
cursor.close()

("Cortador azulejo 35' (90 cm) gran formato c/maletín, Expert",)
('Lija de agua grano 100 de carburo de silicio, Truper',)
("Broca HSS 11/64' Trugold para metal, Truper Expert",)
("Carda copa 5' alambre ondulado fino, eje 5/8'-11, Truper",)
("Caja para herramienta de 20' industrial, broches metálicos",)
("Azadón para niños con mango de 42', Truper",)
("Disco Tipo 41 de 7' x 1.8 mm corte fino acero inox, Sable",)
("Cortador azulejo 47' (120 cm) gran formato maletín, Expert",)
('Bolsa con 2 carbones de repuesto para CM-14N, Truper',)
('Línea de vida 1.8m cinta políéster, absorbedor impacto, ANSI',)
("Broca HSS 7/32' Trugold para metal, Truper Expert",)
("Broca HSS 27/64' Trugold para metal, Truper Expert",)
("Broca HSS 17/64' Trugold para metal, Truper Expert",)
("Pala redonda # 4 T-2000, mango largo 45', Truper",)
('Escalera de tijera tipo II 7 peldaños y bandeja, Truper',)
("Caja para herramienta de 22' con compartimentos, Truper",)
("Bolsa c/2\xa0abrazaderas\xa0sin fin\xa0# 10, 3/4 - 

In [3]:
def invoke(item_1, item_2):
    input_text = f"""You are going to play the role of an assistant at a hardware store and you are going to help a
customer find the item that they are looking for. You asked you co-worker to bring you the item that most resembles the
customer's description. Now, you must determine if they brought the correct item or not.

First, you must determine what type of item the customer is looking for based on the description. For example, if the
customer asks for "Abrazadera #10 (19-27mm) 3/4 - 1-1/16 c/10pz AB10 Fiero," you know they are looking for an
"abrazadera".

If the customer mentions any measurement/caliber/length/gauge, you must also make sure that they are the same. In the
past example, you should make sure the item is #10 and 3/4 - 1 1/16 inches, or 19-27 mm.

Lastly, if the customer mentions a count, you must also make sure to pick an item that contains the same count. The
past examples mentions "10 pz," so make sure to give a customer an item with 10 pieces.

The following is an example of products that are not equivalent:
"Remaches con espiga 6.4x15.9mm bolsa con 50 pzas" and "Remache diam 3/16 agarre 1/2 c/50pz R68B Fiero"
Notice how in this example, the type of item is the same, but the size is different.

The following is an example of products that are equivalent:
"Rotomartillo 1/2' 550 W Pretul" and "Rotomartillo 1/2 550w ROTO1/2P6 Pretul"

The customer is looking for the item with the following description:
"{item_1}"

Your co-worker brought the following item:
"{item_2}"

Are they the same item? Respond with a confidence score (0.0 - 1.0) that represents how likely it is that the item is
the same as the item that the customer is looking for."""

    body = json.dumps({"inputText": input_text, "textGenerationConfig": {"maxTokenCount": 10}})
    modelId = "amazon.titan-text-premier-v1:0"
    accept = "application/json"
    contentType = "application/json"

    try:
        response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    except:
        time.sleep(5)
        response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    responsebody = json.loads(response.get("body").read())
    return responsebody.get("results")[0].get("outputText")

def query_clave(claves):
    if claves == []:
        return []

    ids = ''
    for clave in claves:
        ids += f"'{clave}',"
    ids = ids.removesuffix(',')
    query = f"""SELECT
        articulos.nombre,
        STRING_AGG(claves.clave, ', ') AS claves
    FROM
        public.articulos_dev__articulos AS articulos
    LEFT JOIN
        public.articulos_dev__articulo_clave AS claves
    ON
        articulos.id = claves.articulo_id
    WHERE
        articulos.empresa_id = 279
    AND
        claves.clave IN ({ids})
    GROUP BY
        articulos.nombre;"""

    connection = psycopg2.connect(dbname='*****', user='*****', password='*****', host='*****', port='*****', sslmode='*****')
    cursor = connection.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    cursor.close()
    return results

def query_nombre(nombre):
    query = f"""SELECT nombre
    FROM public.articulos_dev__articulos
    WHERE empresa_id = 279
    AND difference(nombre, '{nombre}') > 2
    ORDER BY levenshtein(nombre, '{nombre}')
    LIMIT 5;"""

    connection = psycopg2.connect(dbname='*****', user='*****', password='*****', host='*****', port='*****', sslmode='*****')
    cursor = connection.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    cursor.close()  
    return results

def buscar(nombre, claves=[]):
    query_results = query_clave(claves)

    if(query_results != []):
        score = float(invoke(nombre, query_results[0][0]))
        if score >= 0.8:
            return {"articulo": query_results[0][0], "resultado": "seguro", "score": score}
        if score >= 0.5:
            return {"articulo": query_results[0][0], "resultado": "probable", "score": score}
        return {"articulo": query_results[0][0], "resultado": "vacio", "score": score}

    query_results = query_nombre(nombre)
    if(query_results == []):
        return {"articulo": "vacio", "resultado": "vacio", "score": 0.0}

    max_score = 0.0
    max_score_nombre = ''
    for articulo in query_results:
        score = float(invoke(nombre, articulo[0]))
        if score > max_score:
            max_score = score
            max_score_nombre = articulo[0]
    if max_score >= 0.9:
        return {"articulo": max_score_nombre, "resultado": "seguro", "score": max_score}
    if max_score >= 0.6:
        return {"articulo": max_score_nombre, "resultado": "probable", "score": max_score}
    return {"articulo": max_score_nombre, "resultado": "vacio", "score": max_score}

In [4]:
print(buscar("Clavija y contacto de hule, bolsa con 2 pzas, Volteck"))

{'articulo': 'Clavija y contacto de hule, bolsa con 2 pzas, Volteck', 'resultado': 'seguro', 'score': 1.0}


In [ ]:
with open("./data/claves-com.csv", "r") as f:
    items = f.read().split('\n')
items.pop(0)
items.pop(0)
items = list(map(lambda x: x.split(','), items))

vis = []
csv = 'INPUT,TRUPER,articulo,score,resultado,articulo,score,resultado\n'
for i in range(200):
    if i % 30 == 0:
        print(i)

    random_index = random.randint(0, len(items) - 1)
    while random_index in vis:
        random_index = random.randint(0, len(items) - 1)
    vis.append(random_index)

    item_truper = items[random_index][1].replace("'", "''")
    item_nombre = items[random_index][3].replace("'", "''")
    item_claves = items[random_index][0].split(' ; ')

    results_con_clave = buscar(item_nombre, item_claves)
    results_sin_clave = buscar(item_nombre)

    csv += f"{item_nombre},{item_truper},{results_con_clave['articulo'].replace(',', ' ')},{results_con_clave['score']},{results_con_clave['resultado']},{results_sin_clave['articulo'].replace(',', ' ')},{results_sin_clave['score']},{results_sin_clave['resultado']}\n"

with open("./data/busqueda.csv", "w+") as f:
    f.write(csv)
 
print(".")

0
30
60
90
120
150
180


In [26]:
print(csv)

INPUT,TRUPER,articulo,score,resultado,articulo,score,resultado
Faja con tirantes ajust.s de malla extra grande faja-xlv tru,Faja ventilada con tirantes talla extra-grande,Faja lumbar de malla transpirable  talla XG  Truper,0.8,seguro,Faja con tirantes broche sencillo gde By Lack,1.0,seguro
Carretilla 55 llanta ref bas/tub cat-55nd truper,Carretilla 5 ft3 angosta llanta neumatica reforzada,vacio,0.0,vacio,Carretilla 5 ft3 Imponchable  bastidor tubular  Truper,1.0,seguro
Foco kripton rep p/lint 2aa 2xblist fo-li2aa truper,FOCO KRIPTON REP P/LINTERNA 2AA 2XBLIST,Rack giratorio para palas y bieldos  Truper,1.0,seguro,Rack giratorio para palas y bieldos  Truper,1.0,seguro
Candado metal 30mm ll/trad g/corto laton caja CH30P Hermex,Candado de hierro 30mm gancho corto caja Hermex Basic,Candado de hierro color latón 30 mm  corto  caja  Basic,0.8,seguro,Candado metal 30mm ll/trad g/corto laton caja 16911 Dexter,1.0,seguro
Llave p/tubo stilson 08 sti-8p pretul,Llave para tubo Stilson 8'' Pretul,L